In [1]:
%cd ../src

/Users/mikhail_zybin/food-order-chat-bot/src


In [2]:
import chatbot
import openai

In [3]:
def read_scenario(scenario_id: int) -> list[dict[str, str]]:
    with open(f"../evaluator_scenarios/scenario{scenario_id}.txt", "r") as fin:
        scenario = fin.readlines()

    # This is to remove the commentaries
    while not scenario[0].startswith("Chatbot"):
        scenario = scenario[1:]
    
    messages = []
    current_replica = "Please help me to order food"
    chatbot_speaks = True
    for line in scenario:
        if line.startswith("You"):
            messages.append({"role": "assistant", "content": current_replica.strip()})
            chatbot_speaks = False
            current_replica = line[5:]
        elif line.startswith("Chatbot"):
            messages.append({"role": "user", "content": current_replica.strip()})
            chatbot_speaks = True
            current_replica = line[9:]
        else:
            current_replica += line
    
    return messages

In [4]:
system_prompt = chatbot.initialize_system_prompt()

In [5]:
messages = [{"role": "system", "content": system_prompt}] + read_scenario(1)

In [6]:
api_base = "https://llama3-1-8b-api.llm.lab.epam.com/v1"
model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
client = openai.OpenAI(api_key="***REMOVED***", base_url=api_base)


In [7]:
from loguru import logger

logger.disable("chatbot")
for i in range(2, len(messages), 2):
    predicted_message = chatbot.get_next_ai_message(messages[:i], model, client)
    ground_truth = messages[i]["content"]
    print("-"*20 + "predicted_message" + "-"*20)
    print(predicted_message)
    print("-"*20 + "ground_truth" + "-"*20)
    print(ground_truth)
    print()

APITimeoutError: Request timed out.